In [2]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# Corrected download URL and filename
!wget -v https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!ls # Add ls to verify download
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

import os
# Corrected SPARK_HOME path
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,152 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataProject").getOrCreate()


In [9]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

df = spark.read.csv("ml-25m/ratings.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)


--2025-07-28 15:41:45--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip.2’

ml-25m.zip.2        100%[===================>] 249.84M  79.2MB/s    in 3.4s    

2025-07-28 15:41:49 (73.2 MB/s) - ‘ml-25m.zip.2’ saved [261978986/261978986]

Archive:  ml-25m.zip
replace ml-25m/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/tags.csv         
replace ml-25m/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/links.csv        
replace ml-25m/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/README.txt       
replace ml-25m/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/ratings.csv      y
y
y

replace ml-25m/genome-tags.csv? [y]e

In [11]:
# Top 10 Most Rated Movies
df.groupBy("movieId") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(10)


+-------+-----+
|movieId|count|
+-------+-----+
|    356|81491|
|    318|81482|
|    296|79672|
|    593|74127|
|   2571|72674|
|    260|68717|
|    480|64144|
|    527|60411|
|    110|59184|
|   2959|58773|
+-------+-----+
only showing top 10 rows



In [12]:
# Top 10 Highest Rated Movies (with >1000 ratings)
from pyspark.sql.functions import avg, count

df.groupBy("movieId") \
  .agg(avg("rating").alias("avg_rating"), count("*").alias("total_ratings")) \
  .filter("total_ratings > 1000") \
  .orderBy("avg_rating", ascending=False) \
  .show(10)

+-------+------------------+-------------+
|movieId|        avg_rating|total_ratings|
+-------+------------------+-------------+
| 171011| 4.483096085409253|         1124|
| 159817| 4.464796794504865|         1747|
|    318| 4.413576004516335|        81482|
| 170705| 4.398598820058997|         1356|
|    858| 4.324336165187245|        52498|
|     50| 4.284353213163313|        55366|
|   1221|4.2617585117585115|        34188|
|   2019|  4.25476920775043|        13367|
|    527| 4.247579083279535|        60411|
|   1203| 4.243014062405697|        16569|
+-------+------------------+-------------+
only showing top 10 rows



In [13]:
# Ratings Count per Rating Value
df.groupBy("rating") \
  .count() \
  .orderBy("rating") \
  .show()


+------+-------+
|rating|  count|
+------+-------+
|   0.5| 393068|
|   1.0| 776815|
|   1.5| 399490|
|   2.0|1640868|
|   2.5|1262797|
|   3.0|4896928|
|   3.5|3177318|
|   4.0|6639798|
|   4.5|2200539|
|   5.0|3612474|
+------+-------+



In [16]:
# Create a Temp View and Run SQL
df.createOrReplaceTempView("ratings")

# Get average rating for movieId = 1
spark.sql("""
    SELECT movieId, AVG(rating) AS avg_rating
    FROM ratings
    WHERE movieId = 1
    GROUP BY movieId
""").show()

+-------+-----------------+
|movieId|       avg_rating|
+-------+-----------------+
|      1|3.893707794587238|
+-------+-----------------+

